In [1]:
import pandas as pd
import os
import csv
import sys
import numpy as np

csv.field_size_limit(100000000)

131072

## Processing https://www.kaggle.com/jruvika/fake-news-detection Dataset into Combined Data

In [4]:
from urllib.parse import urlparse

jruvika_csv = pd.read_csv('./work/Datasets/Kaggle/Dataset_1/data.csv', engine='python')
jruvika_csv.columns = ['url', 'title', 'content', 'label']
jruvika_csv['label'] = np.where(jruvika_csv['label'] == 1, 'reliable', 'fake')

# Extracting out domain from URL
def extract_domain(url):
    partialDomain = urlparse(url).netloc
    return partialDomain.replace("www.", "")
jruvika_csv['domain'] = jruvika_csv['url'].apply(lambda x: extract_domain(x))

jruvika_csv

# Saving to CSV
jruvika_csv.to_csv('./work/Datasets/Combined_Dataset/combined_data.csv', mode='a', index=False)

print('Done')

Done


## Processing LIAR dataset
### Some references to: https://towardsdatascience.com/identifying-fake-news-the-liar-dataset-713eca8af6ac

In [5]:
liar_csv = pd.read_table('./work/Datasets/liar_dataset/train.tsv', names = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
                                            'barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'venue'])

liar_csv = liar_csv.drop(columns=['id', 'subject', 'job', 'state', 'party', 'barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'venue'])

liar_label_dict = {
    "pants-fire" : 'fake', 
    "false" : 'fake', 
    "barely-true" : 'fake', 
    "half-true" : 'fake', 
    "mostly-true" : 'reliable', 
    "true" : 'reliable'
}
liar_csv['label'] = liar_csv['label'].apply(lambda x: liar_label_dict[x])


liar_csv.to_csv('./work/Datasets/Combined_Dataset/truncated_liar_dataset.csv', mode='a', index=False)
print("Done")

Done


## Processing Fakenewscorpus Datas into Combined Data

In [3]:
fakenewscorpus_csv_iter = pd.read_csv('./work/Datasets/FakeNewsCorpus/news_cleaned_2018_02_13.csv', engine='python', iterator=True, chunksize=100000, encoding='utf-8', header=0, error_bad_lines=False)

label_dict = {
    'political': 'reliable',
    'bias': 'fake',
    'unreliable': 'fake',
    'reliable': 'reliable',
    'fake': 'fake'
}

for single_df in fakenewscorpus_csv_iter:
    single_df = single_df[(single_df.type == 'fake') | (single_df.type == 'reliable') | (single_df.type == 'political') | (single_df.type == 'bias') | (single_df.type == 'unreliable')]

    single_df.dropna()
    single_df = single_df.rename({'type': 'label', 'authors': 'author'}, axis='columns')
    single_df = single_df.drop(columns=['id', 'scraped_at', 'inserted_at', 'updated_at', 'keywords', 'meta_keywords', 'meta_description', 'tags', 'summary'])


    single_df['label'] = single_df['label'].apply(lambda x: label_dict[x])
    single_df.to_csv('./work/Datasets/Combined_Dataset/truncated_corpus.csv', mode='a')

print("FINSIHED WITH CORPUS DATA")

FINSIHED WITH CORPUS DATA


# Please send help. Preprocessing data is a pain

In [2]:
kaggle_csv = pd.read_csv('./work/Datasets/Combined_Dataset/combined_data.csv')
kaggle_csv

,url,title,content,label,domain
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,reliable,bbc.com
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",reliable,reuters.com
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,reliable,nytimes.com
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,reliable,reuters.com
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",reliable,cnn.com
...,...,...,...,...,...
8014,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,fake,beforeitsnews.com
8015,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,fake,beforeitsnews.com
8016,https://www.activistpost.com/2017/09/ron-paul-...,"Ron Paul on Trump, Anarchism & the AltRight",NaN,fake,activistpost.com
8017,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,reliable,reuters.com


In [3]:
combined_df = pd.concat([kaggle_csv])
combined_df

,url,title,content,label,domain
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,reliable,bbc.com
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",reliable,reuters.com
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,reliable,nytimes.com
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,reliable,reuters.com
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",reliable,cnn.com
...,...,...,...,...,...
8014,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,fake,beforeitsnews.com
8015,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,fake,beforeitsnews.com
8016,https://www.activistpost.com/2017/09/ron-paul-...,"Ron Paul on Trump, Anarchism & the AltRight",NaN,fake,activistpost.com
8017,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,reliable,reuters.com


In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import swifter
import re

ps = PorterStemmer()

def clean_content(text):
    text = re.sub(r"http\S+", "", text)

    # Tokenize and Remove non alphanumeric characters
    tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    tokens = tokenizer.tokenize(text)
    allalpha_tokens = [word for word in tokens if word.isalpha()]

    # Porter stemmer back to basic english words
    words = [ps.stem(word) for word in allalpha_tokens if not word in stopwords.words('english')]
    words = ' '.join(words)
    return words 
    

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
combined_content_df = combined_df.drop(columns=['url', 'title', 'domain'])
combined_content_df = combined_content_df.reset_index(drop=True)
combined_content_df = combined_content_df.dropna()

combined_content_df['content'] = combined_content_df['content'].swifter.apply(lambda x: clean_content(x))

combined_content_df = combined_content_df.dropna()
combined_content_df.reset_index(drop=True, inplace=True)
combined_content_df.to_csv('./work/Datasets/Combined_Dataset/processed_content_large.csv', mode='a', index=False)

combined_content_df

Pandas Apply:   0%|          | 0/7977 [00:00<?, ?it/s]

,content,label
0,imag copyright getti imag on sunday morn donal...,reliable
1,london flag fli comedi vietnam war veteran res...,reliable
2,the feud broke public view last week mr corker...,reliable
3,mexico citi egypt cheiron hold limit right par...,reliable
4,countri singer jason aldean perform la vega sh...,reliable
...,...,...
7972,vietnam is great danger you must publish tell ...,fake
7973,trend watch reader think stori fact add two ce...,fake
7974,trump jr is soon to give a speech for reader t...,fake
7975,shanghai china said plan accept data oversea c...,reliable


In [11]:
fakenewscorpus_df_iter = pd.read_csv('./work/Datasets/Combined_Dataset/truncated_corpus.csv', engine='python', usecols=['label', 'content'], iterator=True, chunksize=200000, encoding='utf-8')

# test = fakenewscorpus_df_iter.get_chunk(1000)

for single_df in fakenewscorpus_df_iter:
    single_df = single_df.dropna()
    single_df['content'] = single_df['content'].swifter.apply(lambda x: clean_content(x))
    single_df = single_df.dropna()
    single_df.to_csv('./work/Datasets/Combined_Dataset/processed_content_large.csv', mode='a', index=False, header=False)

print("Done concating to CSV file")

Pandas Apply:   0%|          | 0/200000 [00:00<?, ?it/s]